# [Using Environment Variables in Python for App Configuration and Secrets](https://www.doppler.com/blog/environment-variables-in-python)

In [27]:
from os import environ
import sys

In [28]:
environ

environ{'ALLUSERSPROFILE': 'C:\\ProgramData',
        'APPDATA': 'C:\\Users\\Igor\\AppData\\Roaming',
        'COMMONPROGRAMFILES': 'C:\\Program Files\\Common Files',
        'COMMONPROGRAMFILES(X86)': 'C:\\Program Files (x86)\\Common Files',
        'COMMONPROGRAMW6432': 'C:\\Program Files\\Common Files',
        'COMPUTERNAME': 'Z170A',
        'COMSPEC': 'C:\\Windows\\system32\\cmd.exe',
        'DRIVERDATA': 'C:\\Windows\\System32\\Drivers\\DriverData',
        'HOMEDRIVE': 'C:',
        'HOMEPATH': '\\Users\\Igor',
        'INTELLIJ IDEA COMMUNITY EDITION': 'C:\\Program Files\\JetBrains\\IntelliJ IDEA Community Edition\\bin;',
        'LOCALAPPDATA': 'C:\\Users\\Igor\\AppData\\Local',
        'LOGONSERVER': '\\\\Z170A',
        'NUMBER_OF_PROCESSORS': '4',
        'OS': 'Windows_NT',
        'PATH': 'C:\\Program Files\\Common Files\\Oracle\\Java\\javapath;C:\\Program Files\\Python39\\;C:\\Program Files\\Python39\\Scripts\\;C:\\Program Files\\Python38\\;C:\\Program Files\\Python38\

## How to get a Python environment variable
Environment variables in Python are accessed using the `os.environ` object.

The `os.environ` object seems like a dictionary but is different as values may only be strings, plus it's not serializable to JSON.

You've got a few options when it comes to referencing the `os.environ` object:

In [29]:
# 1. Standard way
import os
os.environ['USERNAME']

'Igor'

In [30]:
# 2. Import just the environ object
from os import environ
environ['USERNAME']

'Igor'

In [31]:
# 3. Rename the `environ` to env object for more concise code
from os import environ as env
env['USERNAME']

'Igor'

Accessing a specific environment variable in Python can be done in one of three ways, depending upon what should happen if an environment variable does not exist.

Let's explore with some examples.

### Option 1: Required with no default value
If your app should crash when an environment variable is not set, then access it directly:

In [32]:
print(os.environ['HOMEPATH'])

\Users\Igor


In [33]:
try:
    print(os.environ['DOES_NOT_EXIST'])
except KeyError as e:
    exc_type, exc_value, exc_traceback = sys.exc_info()
    print(f"{exc_type}: {exc_value}")

<class 'KeyError'>: 'DOES_NOT_EXIST'


For example, an application should fail to start if a required environment variable is not set, and a default value can't be provided, e.g. a database password.

If instead of the default `KeyError` exception being raised (which doesn't communicate why your app failed to start), you could capture the exception and print out a helpful message:

In [34]:
import os
import sys

# Ensure all required environment variables are set
try:  
    os.environ['API_KEY']
except KeyError: 
    print('[error]: `API_KEY` environment variable required')
#     sys.exit(1)

[error]: `API_KEY` environment variable required


### Option 2: Required with default value
You can have a default value returned if an environment variable doesn't exist by using the `os.environ.get` method and supplying the default value as the second parameter:

In [35]:
# If HOSTNAME doesn't exist, presume local development and return localhost
print(os.environ.get('HOSTNAME', 'localhost'))

localhost


If the variable doesn't exist and you use `os.environ.get` without a default value, `None` is returned

In [36]:
print(os.environ.get('NO_VAR_EXISTS') == None)

True


### Option 3: Conditional logic if value exists
You may need to check if an environment variable exists, but don't necessarily care about its value. For example, your application can be put in a "Debug mode" if the `DEBUG` environment variable is set.

You can check for just the existence of an environment variable:

In [37]:
if 'DEBUG' in os.environ:
    print('[info]: app is running in debug mode')

Or check to see it matches a specific value:

In [38]:
if os.environ.get('DEBUG') == 'True':
    print('[info]: app is running in debug mode')

## How to set a Python environment variable
Setting an environment variable in Python is the same as setting a key on a dictionary:

In [39]:
os.environ['TESTING'] = 'True'
os.environ.get('TESTING')

'True'

What makes os.environ different to a standard dictionary, is that only string values are allowed:

In [40]:
try:
    os.environ['TESTING'] = True
except TypeError as e:
    exc_type, exc_value, exc_traceback = sys.exc_info()
    print(f"{exc_type}: {exc_value}")

<class 'TypeError'>: str expected, not bool


In most cases, your application will only need to get environment variables, but there are use cases for setting them as well.

For example, constructing a `DB_URL` environment variable on application start-up using `DB_HOST`, `DB_PORT`, `DB_USER`, `DB_PASSWORD`, and `DB_NAME` environment variables:

In [41]:
os.environ.setdefault('DB_USER', 'inside DB_USER'),
os.environ.setdefault('DB_PASSWORD', 'inside DB_PASSWORD')
os.environ.setdefault('DB_HOST', 'inside DB_HOST')
os.environ.setdefault('DB_PORT', 'inside DB_PORT')
os.environ.setdefault('DB_NAME', 'inside DB_NAME')

try:
    os.environ['DB_URL'] = 'psql://{user}:{password}@{host}:{port}/{name}'.format(
      user=os.environ['DB_USER'],
      password=os.environ['DB_PASSWORD'],
      host=os.environ['DB_HOST'],
      port=os.environ['DB_PORT'],
      name=os.environ['DB_NAME']
    )
    print(os.environ.get('DB_URL'))
except KeyError as e:
    exc_type, exc_value, exc_traceback = sys.exc_info()
    print(f"{exc_type}: {exc_value}")

psql://inside DB_USER:inside DB_PASSWORD@inside DB_HOST:inside DB_PORT/inside DB_NAME


Another example is setting a variable to a default value based on the value of another variable:

In [42]:
# Set DEBUG and TESTING to 'True' if ENV is 'development'
try:
    if os.environ.get('ENV') == 'development':
        os.environ.setdefault('DEBUG', 'True') # Only set to True if DEBUG not set
        os.environ.setdefault('TESTING', 'True') # Only set to True if TESTING not set
except KeyError as e:
    exc_type, exc_value, exc_traceback = sys.exc_info()
    print(f"{exc_type}: {exc_value}")

## How to delete a Python environment variable
If you need to delete a Python environment variable, use the `os.environ.pop` function:

In [43]:
os.environ.setdefault('MY_KEY', 'inside MY_KEY')
print(os.environ.get('MY_KEY'))
print(os.environ.pop('MY_KEY'))
print(os.environ.get('MY_KEY'))

inside MY_KEY
inside MY_KEY
None


To extend our `DB_URL` example above, you may want to delete the other `DB_` prefixed fields to ensure the only way the app can connect to the database is via `DB_URL`:

Another example is deleting an environment variable once it is no longer needed:

In [44]:
def auth_api(api_key: str) -> None:
    print(f"Do smth. with an {api_key}.")

os.environ.setdefault('API_KEY', 'API key')
auth_api(os.environ['API_KEY']) # Use API_KEY
os.environ.pop('API_KEY') # Delete API_KEY as it's no longer needed
print(os.environ.get('API_KEY'))

Do smth. with an API key.
None


## How to list Python environment variables
To view all environment variables:

In [45]:
print(os.environ)

environ({'ALLUSERSPROFILE': 'C:\\ProgramData', 'APPDATA': 'C:\\Users\\Igor\\AppData\\Roaming', 'COMMONPROGRAMFILES': 'C:\\Program Files\\Common Files', 'COMMONPROGRAMFILES(X86)': 'C:\\Program Files (x86)\\Common Files', 'COMMONPROGRAMW6432': 'C:\\Program Files\\Common Files', 'COMPUTERNAME': 'Z170A', 'COMSPEC': 'C:\\Windows\\system32\\cmd.exe', 'DRIVERDATA': 'C:\\Windows\\System32\\Drivers\\DriverData', 'HOMEDRIVE': 'C:', 'HOMEPATH': '\\Users\\Igor', 'INTELLIJ IDEA COMMUNITY EDITION': 'C:\\Program Files\\JetBrains\\IntelliJ IDEA Community Edition\\bin;', 'LOCALAPPDATA': 'C:\\Users\\Igor\\AppData\\Local', 'LOGONSERVER': '\\\\Z170A', 'NUMBER_OF_PROCESSORS': '4', 'OS': 'Windows_NT', 'PATH': 'C:\\Program Files\\Common Files\\Oracle\\Java\\javapath;C:\\Program Files\\Python39\\;C:\\Program Files\\Python39\\Scripts\\;C:\\Program Files\\Python38\\;C:\\Program Files\\Python38\\Scripts\\;C:\\Windows\\system32;C:\\Windows;C:\\Windows\\System32\\Wbem;C:\\Windows\\System32\\WindowsPowerShell\\v1.0

The output of this command is difficult to read though because it's printed as one huge dictionary.

A better way, is to create a convenience function that converts `os.environ` to an actual dictionary so we can serialize it to `JSON` for pretty-printing:

In [46]:
import os
import json

def print_env():
    print(json.dumps({**{}, **os.environ}, indent=2))

print_env()

{
  "ALLUSERSPROFILE": "C:\\ProgramData",
  "APPDATA": "C:\\Users\\Igor\\AppData\\Roaming",
  "COMMONPROGRAMFILES": "C:\\Program Files\\Common Files",
  "COMMONPROGRAMFILES(X86)": "C:\\Program Files (x86)\\Common Files",
  "COMMONPROGRAMW6432": "C:\\Program Files\\Common Files",
  "COMPUTERNAME": "Z170A",
  "COMSPEC": "C:\\Windows\\system32\\cmd.exe",
  "DRIVERDATA": "C:\\Windows\\System32\\Drivers\\DriverData",
  "HOMEDRIVE": "C:",
  "HOMEPATH": "\\Users\\Igor",
  "INTELLIJ IDEA COMMUNITY EDITION": "C:\\Program Files\\JetBrains\\IntelliJ IDEA Community Edition\\bin;",
  "LOCALAPPDATA": "C:\\Users\\Igor\\AppData\\Local",
  "LOGONSERVER": "\\\\Z170A",
  "NUMBER_OF_PROCESSORS": "4",
  "OS": "Windows_NT",
  "PATH": "C:\\Program Files\\Common Files\\Oracle\\Java\\javapath;C:\\Program Files\\Python39\\;C:\\Program Files\\Python39\\Scripts\\;C:\\Program Files\\Python38\\;C:\\Program Files\\Python38\\Scripts\\;C:\\Windows\\system32;C:\\Windows;C:\\Windows\\System32\\Wbem;C:\\Windows\\System32

Or you can simply use `pprint`:

In [47]:
from pprint import pprint

pprint(dict(environ))

{'ALLUSERSPROFILE': 'C:\\ProgramData',
 'APPDATA': 'C:\\Users\\Igor\\AppData\\Roaming',
 'CLICOLOR': '1',
 'COMMONPROGRAMFILES': 'C:\\Program Files\\Common Files',
 'COMMONPROGRAMFILES(X86)': 'C:\\Program Files (x86)\\Common Files',
 'COMMONPROGRAMW6432': 'C:\\Program Files\\Common Files',
 'COMPUTERNAME': 'Z170A',
 'COMSPEC': 'C:\\Windows\\system32\\cmd.exe',
 'DB_HOST': 'inside DB_HOST',
 'DB_NAME': 'inside DB_NAME',
 'DB_PASSWORD': 'inside DB_PASSWORD',
 'DB_PORT': 'inside DB_PORT',
 'DB_URL': 'psql://inside DB_USER:inside DB_PASSWORD@inside DB_HOST:inside '
           'DB_PORT/inside DB_NAME',
 'DB_USER': 'inside DB_USER',
 'DRIVERDATA': 'C:\\Windows\\System32\\Drivers\\DriverData',
 'GIT_PAGER': 'cat',
 'HOMEDRIVE': 'C:',
 'HOMEPATH': '\\Users\\Igor',
 'INTELLIJ IDEA COMMUNITY EDITION': 'C:\\Program Files\\JetBrains\\IntelliJ '
                                    'IDEA Community Edition\\bin;',
 'IPY_INTERRUPT_EVENT': '2748',
 'JPY_INTERRUPT_EVENT': '2748',
 'JPY_PARENT_PID': '189

## Using a .env file for Python environment variables
As an application grows in size and complexity, so does the number of environment variables.

Many projects experience growing pains when using environment variables for app config and secrets because there is no clear and consistent strategy for how to manage them, particularly when deploying to multiple environments.

A simple (but not easily scalable) solution is to use a `.env` file to contain all of the  variables for a specific environment.

Then you would use a Python library such as `python-dotenv` to parse the `.env` file and populate the `os.environ` object.

Install the `python-dotenv` library:

In [57]:
# install python-dotenv
# %run -m pip install python-dotenv

Now save the below to a file named `.env` (note how it's the same syntax for setting a variable in the shell):

In [75]:
s = f'API_KEY="357A70FF-BFAA-4C6A-8289-9831DDFB2D3D"\nHOSTNAME="0.0.0.0"\nPORT="8080"'

with open('.env', 'w') as f:
    f.write(s)

In [64]:
# Rename `os.environ` to `env` for nicer code
from os import environ as env

from dotenv import load_dotenv
load_dotenv()

print('API_KEY:  {}'.format(env['API_KEY']))
print('HOSTNAME: {}'.format(env['HOSTNAME']))
print('PORT:     {}'.format(env['PORT']))

API_KEY:  357A70FF-BFAA-4C6A-8289-9831DDFB2D3D
HOSTNAME: 0.0.0.0
PORT:     8080


In [65]:
env

environ{'ALLUSERSPROFILE': 'C:\\ProgramData',
        'APPDATA': 'C:\\Users\\Igor\\AppData\\Roaming',
        'COMMONPROGRAMFILES': 'C:\\Program Files\\Common Files',
        'COMMONPROGRAMFILES(X86)': 'C:\\Program Files (x86)\\Common Files',
        'COMMONPROGRAMW6432': 'C:\\Program Files\\Common Files',
        'COMPUTERNAME': 'Z170A',
        'COMSPEC': 'C:\\Windows\\system32\\cmd.exe',
        'DRIVERDATA': 'C:\\Windows\\System32\\Drivers\\DriverData',
        'HOMEDRIVE': 'C:',
        'HOMEPATH': '\\Users\\Igor',
        'INTELLIJ IDEA COMMUNITY EDITION': 'C:\\Program Files\\JetBrains\\IntelliJ IDEA Community Edition\\bin;',
        'LOCALAPPDATA': 'C:\\Users\\Igor\\AppData\\Local',
        'LOGONSERVER': '\\\\Z170A',
        'NUMBER_OF_PROCESSORS': '4',
        'OS': 'Windows_NT',
        'PATH': 'C:\\Program Files\\Common Files\\Oracle\\Java\\javapath;C:\\Program Files\\Python39\\;C:\\Program Files\\Python39\\Scripts\\;C:\\Program Files\\Python38\\;C:\\Program Files\\Python38\

While `.env` files are simple and easy to work with at the beginning, they also cause a new set of problems such as:

* How to keep .env files in-sync for every developer in their local environment?
* If there is an outage due to misconfiguration, accessing the container or VM directly in order to view the contents of the .env may be required for troubleshooting.
* How do you generate a `.env` file for a CI/CD job such as GitHub Actions without committing the .env file to the repository?
* If a mix of environment variables and a `.env` file is used, the only way to determine the final configuration values could be by introspecting the application.
* Onboarding a developer by sharing an unencrypted `.env` file with potentially sensitive data in a chat application such as Slack could pose security issues.
These are just some of the reasons why we recommend [moving away from .env files](https://www.doppler.com/blog/goodbye-env-files) and using something like [Doppler](https://www.doppler.com/) instead.

Doppler provides an access controlled dashboard to manage environment variables for every environment with an [easy to use CLI for accessing config and secrets](https://docs.doppler.com/docs/enclave-installation) that works for every language, framework, and platform.

## Centralize application config using a Python data structure
Creating a config specific data structure abstracts away how the config values are set, what fields have default values (if any), and provides a single interface for accessing config values instead of `os.environ` being littered throughout your codebase.

Below is a reasonably full-featured solution that supports:
* Required fields
* Optional fields with defaults
* Type checking and typecasting

The `Config` object exposed in `config.py` is then used below (you can also reside this in the `app.py`):

In [76]:
from config import Config

print('ENV:      {}'.format(Config.ENV))
print('DEBUG:    {}'.format(Config.DEBUG))
print('API_KEY:  {}'.format(Config.API_KEY))
print('HOSTNAME: {}'.format(Config.HOSTNAME))
print('PORT:     {}'.format(Config.PORT))

ENV:      production
DEBUG:    False
API_KEY:  357A70FF-BFAA-4C6A-8289-9831DDFB2D3D
HOSTNAME: 0.0.0.0
PORT:     8080


You can [view this code on GitHub](https://gist.github.com/ryan-blunden/fc9fbaf2da65dd2200fb997bfb0aa365) and if you're after a more full-featured typesafe config solution, then check out the excellent [Pydantic library](https://pydantic-docs.helpmanual.io/).

## Summary
Awesome work! Now you know how to use environment variables in Python for application config and secrets.

In [77]:
from shutil import rmtree

try:
    os.remove('.env')
    os.environ.pop('API_KEY')
    os.environ.pop('HOSTNAME')
    os.environ.pop('PORT')
    rmtree('__pycache__', ignore_errors=True)
    rmtree('.ipynb_checkpoints', ignore_errors=True)
except FileNotFoundError:
    exc_type, exc_value, exc_traceback = sys.exc_info()
    print(f"{exc_type}: {exc_value}")

In [79]:
# unstall python-dotenv
# %run -m pip uninstall --yes python-dotenv

Found existing installation: python-dotenv 0.18.0
Uninstalling python-dotenv-0.18.0:
  Successfully uninstalled python-dotenv-0.18.0
